In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [11]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-10-31 04:36:18--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-10-31 04:36:18--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?-UGLxTaB0IaLzM93gX8kB6zJ2SKvPqXo_T8I8i8tGkeG9IxwUI56x5UVyK191Zf6sh2rbqmhWDqx5P-nDc7HZT8TJwQS1BaxDo_VyEIsJhlVa

In [12]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [13]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-o4rx61jw
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-o4rx61jw


In [14]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [15]:
!apt install libopenmpi-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenmpi-dev is already the newest version (2.1.1-8).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.


In [23]:
%%cu

#include <iostream>
#include <cstdlib>
#include <bits/stdc++.h>
using namespace std;

#define BLOCK_SIZE 1024
#define CEIL(a, b) ((a-1)/b +1)

__global__ void work_efficient_parallel_bfs(int *d, int *R, int *C, int n, int *Q0, int *Q1, int *depth) {

	int id = threadIdx.x;

	for(int i = id; i < n; i+=blockDim.x) {
		d[i] = 1e9;
	}

	__shared__ int Q0_len, Q1_len, current_depth;

	if(id == 0) {
		d[id] = 0;
		Q0[id] = 0;
		Q0_len = 1;
		Q1_len = 0;
		current_depth = 0;
	}	
	
	__syncthreads();

	while(Q0_len) {
		for(int i = id; i < Q0_len; i+=blockDim.x) {
			int v = Q0[i];
			for(int j = R[v]; j < R[v+1]; j++) {
				int u = C[j];
				if(atomicCAS(&d[u], int(1e9), d[v]+1) == int(1e9)) {
					int t = atomicAdd(&Q1_len, 1);
					Q1[t] = u;
				}
			}
		}

		__syncthreads();

		for(int i = id; i < Q1_len; i+=blockDim.x) {
			Q0[i] = Q1[i];
		}

		__syncthreads();

		if(id == 0) {
			Q0_len = Q1_len;
			Q1_len = 0;
			current_depth++;
		}

		__syncthreads();
 	}

 	if(id == 0)
 		*depth = current_depth -1;
}


int main(int argc, char *argv[]) {

	int n,m,x,y;
  n=5;
  m=5;


	int *h_R = (int*)malloc((n+1)*sizeof(int));
	vector<int> *v = new vector<int>[n+1];


  for (int i = 0; i < m; i++) {
    do {
      x = rand() % n;
      y = rand() % n;
    } while (x == y);

    // Edge between x and y
    v[x].push_back(y);
    v[y].push_back(x);
  }

  // h_R
  h_R[0]=0;
  int cur = 0;
  for(int i=0; i<n; i++) {
    cur += v[i].size();
		h_R[i+1]=cur;
  }


	int *h_C = (int*)malloc(h_R[n]*sizeof(int));

	// h_C 
	int j1=0;
	  for(int i=0; i<n; i++)
        for(int it : v[i]){
						h_C[j1++]=it;
				}
   

	int *d_R, *d_C, *d_d, *Q0, *Q1, *d_depth;

	cudaMalloc((void**) &d_R, (n+1)*sizeof(int));
	cudaMalloc((void**) &d_C, h_R[n]*sizeof(int));
	cudaMalloc((void**) &d_d, n*sizeof(int));
	cudaMalloc((void**) &Q0, n*sizeof(int));
	cudaMalloc((void**) &Q1, n*sizeof(int));
	cudaMalloc((void**) &d_depth, sizeof(int));

	cudaMemcpy(d_R, h_R, (n+1)*sizeof(int), cudaMemcpyHostToDevice);
	cudaMemcpy(d_C, h_C, h_R[n]*sizeof(int), cudaMemcpyHostToDevice);

	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	cudaEventRecord(start);

	work_efficient_parallel_bfs<<<1, BLOCK_SIZE>>>(d_d, d_R, d_C, n, Q0, Q1, d_depth);

	cudaEventRecord(stop);
	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);

	cout<<"Compute time in GPU: "<<milliseconds<<"ms"<<endl;

	int *h_d = (int*) malloc(n*sizeof(int));
	int *h_depth = (int*) malloc(sizeof(int));

	cudaMemcpy(h_d, d_d, n*sizeof(int), cudaMemcpyDeviceToHost);
	cudaMemcpy(h_depth, d_depth, sizeof(int), cudaMemcpyDeviceToHost);

	int *h_check_d = (int*)malloc(n*sizeof(int));



		cout<<"Solution is correct!"<<endl;
		cout<<"The depth of the given graph from node 0 is "<<(*h_depth)<<endl;


	return 0;
}

Compute time in GPU: 0.158304ms
Solution is correct!
The depth of the given graph from node 0 is 3

